In [2]:
D.<x> = PolynomialRing(QQ)
R = PolynomialRing(QQ,'x')
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x

def H4(n,m): # this is the definition in the published interpolations paper (2021):
    # def. 8.4
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    #if argument is an S type then 'polynomial' takes no argument.

def H4strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H4(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer


def H6(n,m):
    def base6(n,m):
        jay=J(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(jay-1))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
     #if argument is an S type then 'polynomial' takes no argument.

def H6strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H6(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return ((ddstep1(n,m)-ddstep2(n,m))/1728)

def deltaDaggerStrike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def deltaDagger2(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))

# def deltaDaggerStrike(n,m):
 #   return S(((H4strike(n,3)^3-H6strike(n,3)^2)/1728)).truncate(n+1)
    
def deltaDagger2Strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger2(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr



print("ok")

ok


In [4]:
data=[]
import time
start=time.time()/60
for m in [3..1203]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no5.txt','wb') # H6 series (date typo)
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 100, 0.01920602098107338]
[4, 100, 0.031294867396354675]
[5, 100, 0.05478896573185921]
[6, 100, 0.06679476797580719]
[7, 100, 0.09261558577418327]
[8, 100, 0.11515278741717339]
[9, 100, 0.1471802480518818]
[10, 100, 0.16542823612689972]
[11, 100, 0.19381213560700417]
[12, 100, 0.21792655065655708]
[13, 100, 0.24757971614599228]
[14, 100, 0.2671714536845684]
[15, 100, 0.30094316601753235]
[16, 100, 0.3241902329027653]
[17, 100, 0.35571761801838875]
[18, 100, 0.3774224668741226]
[19, 100, 0.40772466734051704]
[20, 100, 0.43594200536608696]
[21, 100, 0.4705631360411644]
[22, 100, 0.4923041835427284]
[23, 100, 0.5239979177713394]
[24, 100, 0.5523416697978973]
[25, 100, 0.5872596688568592]
[26, 100, 0.610652070492506]
[27, 100, 0.64558345079422]
[28, 100, 0.6746696196496487]
[29, 100, 0.7071546651422977]
[30, 100, 0.7330390885472298]
[31, 100, 0.7652565874159336]
[32, 100, 0.7912880182266235]
[33, 100, 0.8296048678457737]
[34, 100, 0.8542117327451706]
[35, 100, 0.8926257528364658]
[36, 

[277, 100, 10.57899021729827]
[278, 100, 10.616733934730291]
[279, 100, 10.665958669036627]
[280, 100, 10.711722899228334]
[281, 100, 10.757830567657948]
[282, 100, 10.797255549579859]
[283, 100, 10.844126652926207]
[284, 100, 10.891548451036215]
[285, 100, 10.944669216871262]
[286, 100, 10.982225149869919]
[287, 100, 11.032779403030872]
[288, 100, 11.074211217463017]
[289, 100, 11.123413547873497]
[290, 100, 11.161531183868647]
[291, 100, 11.21074603497982]
[292, 100, 11.251755867153406]
[293, 100, 11.298508148640394]
[294, 100, 11.338502682745457]
[295, 100, 11.38787853717804]
[296, 100, 11.428811687976122]
[297, 100, 11.478061117231846]
[298, 100, 11.513078283518553]
[299, 100, 11.562118634581566]
[300, 100, 11.607976898550987]
[301, 100, 11.657102018594742]
[302, 100, 11.692105617374182]
[303, 100, 11.741803716868162]
[304, 100, 11.782439652830362]
[305, 100, 11.833110466599464]
[306, 100, 11.872573036700487]
[307, 100, 11.92022980377078]
[308, 100, 11.965194802731276]
[309, 100, 1

[544, 100, 23.05537459999323]
[545, 100, 23.10810226947069]
[546, 100, 23.152667704969645]
[547, 100, 23.202089853584766]
[548, 100, 23.24814898520708]
[549, 100, 23.302389416843653]
[550, 100, 23.34639683365822]
[551, 100, 23.39860473573208]
[552, 100, 23.44691837206483]
[553, 100, 23.499335803091526]
[554, 100, 23.53941810503602]
[555, 100, 23.59575378522277]
[556, 100, 23.640841580927372]
[557, 100, 23.69316815212369]
[558, 100, 23.73569591715932]
[559, 100, 23.788678850978613]
[560, 100, 23.836712166666985]
[561, 100, 23.89248299971223]
[562, 100, 23.931945085525513]
[563, 100, 23.983011551201344]
[564, 100, 24.0328868329525]
[565, 100, 24.085709899663925]
[566, 100, 24.127086866647005]
[567, 100, 24.182673551142216]
[568, 100, 24.2282429151237]
[569, 100, 24.277479354292154]
[570, 100, 24.322864450514317]
[571, 100, 24.373478300869465]
[572, 100, 24.420765198767185]
[573, 100, 24.474257618188858]
[574, 100, 24.516100604087114]
[575, 100, 24.57145868241787]
[576, 100, 24.6167832352

[813, 100, 36.513850785791874]
[814, 100, 36.55646051838994]
[815, 100, 36.61209665238857]
[816, 100, 36.66290418058634]
[817, 100, 36.71797700226307]
[818, 100, 36.761258233338594]
[819, 100, 36.81943733245134]
[820, 100, 36.87112620100379]
[821, 100, 36.923919167369604]
[822, 100, 36.96869193390012]
[823, 100, 37.022538751363754]
[824, 100, 37.07156543433666]
[825, 100, 37.13042973354459]
[826, 100, 37.17443761974573]
[827, 100, 37.22864678502083]
[828, 100, 37.279553066939116]
[829, 100, 37.332759130746126]
[830, 100, 37.3773522824049]
[831, 100, 37.4321303665638]
[832, 100, 37.47824961692095]
[833, 100, 37.53483986854553]
[834, 100, 37.58262160047889]
[835, 100, 37.63693082332611]
[836, 100, 37.68811336904764]
[837, 100, 37.74401331692934]
[838, 100, 37.787200301885605]
[839, 100, 37.8401162661612]
[840, 100, 37.894710186868906]
[841, 100, 37.946927201002836]
[842, 100, 37.989176120609045]
[843, 100, 38.04569338262081]
[844, 100, 38.09334548562765]
[845, 100, 38.15071699768305]
[84

[1082, 100, 50.55672808364034]
[1083, 100, 50.615631867200136]
[1084, 100, 50.664462730288506]
[1085, 100, 50.72363035008311]
[1086, 100, 50.77153655141592]
[1087, 100, 50.827349215745926]
[1088, 100, 50.87579179927707]
[1089, 100, 50.935299683362246]
[1090, 100, 50.98229593783617]
[1091, 100, 51.0376731492579]
[1092, 100, 51.09356938302517]
[1093, 100, 51.1491308324039]
[1094, 100, 51.19292277097702]
[1095, 100, 51.252806685864925]
[1096, 100, 51.30184135213494]
[1097, 100, 51.357254818081856]
[1098, 100, 51.40353603288531]
[1099, 100, 51.46062905341387]
[1100, 100, 51.5142959356308]
[1101, 100, 51.57187943533063]
[1102, 100, 51.61608266830444]
[1103, 100, 51.67038415372372]
[1104, 100, 51.72193178534508]
[1105, 100, 51.78093338385224]
[1106, 100, 51.82713198661804]
[1107, 100, 51.885798051953316]
[1108, 100, 51.936463836580515]
[1109, 100, 51.99172396957874]
[1110, 100, 52.04082041978836]
[1111, 100, 52.09803866967559]
[1112, 100, 52.147489722818136]
[1113, 100, 52.207221403717995]
[

In [5]:
data=[]
import time
start=time.time()/60
for m in [1204..2403]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no8.txt','wb') # H6 series (date typo)
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[1204, 100, 0.057562053203582764]
[1205, 100, 0.12759896367788315]
[1206, 100, 0.18132508173584938]
[1207, 100, 0.24145756661891937]
[1208, 100, 0.29697979986667633]
[1209, 100, 0.36952798441052437]
[1210, 100, 0.4200063645839691]
[1211, 100, 0.48153189942240715]
[1212, 100, 0.5419398993253708]
[1213, 100, 0.6128267794847488]
[1214, 100, 0.6697229035198689]
[1215, 100, 0.7411110997200012]
[1216, 100, 0.7996407002210617]
[1217, 100, 0.8643246330320835]
[1218, 100, 0.9236241802573204]
[1219, 100, 0.9845627173781395]
[1220, 100, 1.0425902679562569]
[1221, 100, 1.108863815665245]
[1222, 100, 1.1580629646778107]
[1223, 100, 1.2220319658517838]
[1224, 100, 1.282991535961628]
[1225, 100, 1.3523121997714043]
[1226, 100, 1.4030629843473434]
[1227, 100, 1.4677363522350788]
[1228, 100, 1.5247195176780224]
[1229, 100, 1.5858094170689583]
[1230, 100, 1.6472132503986359]
[1231, 100, 1.7111426815390587]
[1232, 100, 1.7707357369363308]
[1233, 100, 1.8346494510769844]
[1234, 100, 1.8813506998121738]
[1

[1465, 100, 14.586096949875355]
[1466, 100, 14.633419267833233]
[1467, 100, 14.692646116018295]
[1468, 100, 14.745531715452671]
[1469, 100, 14.803722981363535]
[1470, 100, 14.856902699917555]
[1471, 100, 14.914555862545967]
[1472, 100, 14.963213484734297]
[1473, 100, 15.022559467703104]
[1474, 100, 15.069543149322271]
[1475, 100, 15.130745597183704]
[1476, 100, 15.185464698821306]
[1477, 100, 15.244460687041283]
[1478, 100, 15.289557415992022]
[1479, 100, 15.350348081439734]
[1480, 100, 15.404812596738338]
[1481, 100, 15.461394812911749]
[1482, 100, 15.511810947209597]
[1483, 100, 15.569805279374123]
[1484, 100, 15.625466734170914]
[1485, 100, 15.689106483012438]
[1486, 100, 15.735900618135929]
[1487, 100, 15.793596670031548]
[1488, 100, 15.847433503717184]
[1489, 100, 15.903528086841106]
[1490, 100, 15.951261699199677]
[1491, 100, 16.015120901167393]
[1492, 100, 16.068950317800045]
[1493, 100, 16.127439733594656]
[1494, 100, 16.179360296577215]
[1495, 100, 16.243417397141457]
[1496, 1

[1724, 100, 29.039174135774374]
[1725, 100, 29.103963434696198]
[1726, 100, 29.151429198682308]
[1727, 100, 29.210947636514902]
[1728, 100, 29.26390326395631]
[1729, 100, 29.32588581740856]
[1730, 100, 29.374509532004595]
[1731, 100, 29.434289816766977]
[1732, 100, 29.487523149698973]
[1733, 100, 29.545981280505657]
[1734, 100, 29.596790302544832]
[1735, 100, 29.657369900494814]
[1736, 100, 29.71269031614065]
[1737, 100, 29.774164952337742]
[1738, 100, 29.82647129893303]
[1739, 100, 29.892765384167433]
[1740, 100, 29.953504033386707]
[1741, 100, 30.01521296426654]
[1742, 100, 30.06557608395815]
[1743, 100, 30.132096767425537]
[1744, 100, 30.186463817954063]
[1745, 100, 30.254824601113796]
[1746, 100, 30.308614052832127]
[1747, 100, 30.371271766722202]
[1748, 100, 30.427498184144497]
[1749, 100, 30.49019406735897]
[1750, 100, 30.539856918156147]
[1751, 100, 30.599206149578094]
[1752, 100, 30.65270296856761]
[1753, 100, 30.710347298532724]
[1754, 100, 30.757909566164017]
[1755, 100, 30.8

[1986, 100, 43.83300777897239]
[1987, 100, 43.8921984359622]
[1988, 100, 43.948360215872526]
[1989, 100, 44.01274196431041]
[1990, 100, 44.06356395035982]
[1991, 100, 44.127431500703096]
[1992, 100, 44.18353195115924]
[1993, 100, 44.24241508170962]
[1994, 100, 44.28920266777277]
[1995, 100, 44.35587563365698]
[1996, 100, 44.408978685736656]
[1997, 100, 44.46830286458135]
[1998, 100, 44.5186318680644]
[1999, 100, 44.578837767243385]
[2000, 100, 44.633899714797735]
[2001, 100, 44.696618065238]
[2002, 100, 44.7478761151433]
[2003, 100, 44.805880934000015]
[2004, 100, 44.862020932137966]
[2005, 100, 44.92300159856677]
[2006, 100, 44.97155239805579]
[2007, 100, 45.03325702995062]
[2008, 100, 45.0876361168921]
[2009, 100, 45.14929233491421]
[2010, 100, 45.20333009958267]
[2011, 100, 45.2621631808579]
[2012, 100, 45.3162395991385]
[2013, 100, 45.379299100488424]
[2014, 100, 45.427330799400806]
[2015, 100, 45.49006414785981]
[2016, 100, 45.546087600290775]
[2017, 100, 45.60580740123987]
[2018,

[2249, 100, 58.88941291719675]
[2250, 100, 58.942737348377705]
[2251, 100, 59.00230550020933]
[2252, 100, 59.057242568582296]
[2253, 100, 59.11958033591509]
[2254, 100, 59.16959862038493]
[2255, 100, 59.23379550129175]
[2256, 100, 59.289454348385334]
[2257, 100, 59.349438052624464]
[2258, 100, 59.39846793562174]
[2259, 100, 59.46237461641431]
[2260, 100, 59.51956846192479]
[2261, 100, 59.58377121761441]
[2262, 100, 59.63651826605201]
[2263, 100, 59.69743676483631]
[2264, 100, 59.75196876749396]
[2265, 100, 59.81798920035362]
[2266, 100, 59.86843879893422]
[2267, 100, 59.92997485026717]
[2268, 100, 59.989935502409935]
[2269, 100, 60.05122008174658]
[2270, 100, 60.10226710140705]
[2271, 100, 60.16546843573451]
[2272, 100, 60.217406284064054]
[2273, 100, 60.276069067418575]
[2274, 100, 60.328201703727245]
[2275, 100, 60.39395923167467]
[2276, 100, 60.450423665344715]
[2277, 100, 60.51473518460989]
[2278, 100, 60.5645705871284]
[2279, 100, 60.62518419697881]
[2280, 100, 60.68533914908767]


In [6]:
data=[]
import time
start=time.time()/60
for m in [2404..3610]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no11.txt','wb') # H6 series (date typo)
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[2404, 100, 0.06253445148468018]
[2405, 100, 0.14007506519556046]
[2406, 100, 0.19933276623487473]
[2407, 100, 0.26878316700458527]
[2408, 100, 0.3328746035695076]
[2409, 100, 0.4002775326371193]
[2410, 100, 0.45265261456370354]
[2411, 100, 0.5169603861868382]
[2412, 100, 0.5782325193285942]
[2413, 100, 0.644251249730587]
[2414, 100, 0.6953175850212574]
[2415, 100, 0.7659838497638702]
[2416, 100, 0.8219972997903824]
[2417, 100, 0.8848879858851433]
[2418, 100, 0.9394984990358353]
[2419, 100, 1.0015518330037594]
[2420, 100, 1.061924796551466]
[2421, 100, 1.127003200352192]
[2422, 100, 1.1794735342264175]
[2423, 100, 1.2414668835699558]
[2424, 100, 1.2997385673224926]
[2425, 100, 1.364034067839384]
[2426, 100, 1.4144965671002865]
[2427, 100, 1.4792805649340153]
[2428, 100, 1.5352302491664886]
[2429, 100, 1.5999496839940548]
[2430, 100, 1.655090481042862]
[2431, 100, 1.7198546677827835]
[2432, 100, 1.772588785737753]
[2433, 100, 1.8379723690450191]
[2434, 100, 1.8886330462992191]
[2435, 10

[2665, 100, 15.638677462935448]
[2666, 100, 15.690031051635742]
[2667, 100, 15.755471032112837]
[2668, 100, 15.81216636300087]
[2669, 100, 15.873961247503757]
[2670, 100, 15.929541748017073]
[2671, 100, 15.989636518061161]
[2672, 100, 16.044465228915215]
[2673, 100, 16.108891487121582]
[2674, 100, 16.16159926354885]
[2675, 100, 16.224738251417875]
[2676, 100, 16.284252636134624]
[2677, 100, 16.345045767724514]
[2678, 100, 16.395701199769974]
[2679, 100, 16.460945185273886]
[2680, 100, 16.518848817795515]
[2681, 100, 16.58188396692276]
[2682, 100, 16.63540694862604]
[2683, 100, 16.698681984096766]
[2684, 100, 16.756234250962734]
[2685, 100, 16.824398152530193]
[2686, 100, 16.876619301736355]
[2687, 100, 16.93864591792226]
[2688, 100, 16.995534747838974]
[2689, 100, 17.056709315627813]
[2690, 100, 17.109601501375437]
[2691, 100, 17.17569973319769]
[2692, 100, 17.23123225197196]
[2693, 100, 17.292892016470432]
[2694, 100, 17.34501838684082]
[2695, 100, 17.411645099520683]
[2696, 100, 17.4

[2925, 100, 31.073787350207567]
[2926, 100, 31.128729719668627]
[2927, 100, 31.189834352582693]
[2928, 100, 31.247343935072422]
[2929, 100, 31.30943663418293]
[2930, 100, 31.362522698938847]
[2931, 100, 31.426640920341015]
[2932, 100, 31.48271745443344]
[2933, 100, 31.5475690998137]
[2934, 100, 31.600197348743677]
[2935, 100, 31.665828719735146]
[2936, 100, 31.72106421738863]
[2937, 100, 31.787007868289948]
[2938, 100, 31.838999036699533]
[2939, 100, 31.901895351707935]
[2940, 100, 31.96594151481986]
[2941, 100, 32.02854213491082]
[2942, 100, 32.079437501728535]
[2943, 100, 32.143777932971716]
[2944, 100, 32.19664936885238]
[2945, 100, 32.261684600263834]
[2946, 100, 32.313447281718254]
[2947, 100, 32.37594786286354]
[2948, 100, 32.43187266588211]
[2949, 100, 32.4976019859314]
[2950, 100, 32.55003033205867]
[2951, 100, 32.61374548450112]
[2952, 100, 32.672433167696]
[2953, 100, 32.734517730772495]
[2954, 100, 32.78520356863737]
[2955, 100, 32.852687649428844]
[2956, 100, 32.90870502963

[3188, 100, 46.874335899949074]
[3189, 100, 46.93926680088043]
[3190, 100, 46.994580037891865]
[3191, 100, 47.05636163428426]
[3192, 100, 47.117810331285]
[3193, 100, 47.180373668670654]
[3194, 100, 47.23133334890008]
[3195, 100, 47.29959240183234]
[3196, 100, 47.35886388644576]
[3197, 100, 47.42101178318262]
[3198, 100, 47.47589238360524]
[3199, 100, 47.54020323231816]
[3200, 100, 47.595000967383385]
[3201, 100, 47.661557883024216]
[3202, 100, 47.71165656670928]
[3203, 100, 47.77454074844718]
[3204, 100, 47.83236473798752]
[3205, 100, 47.89714478701353]
[3206, 100, 47.94907983392477]
[3207, 100, 48.01402298361063]
[3208, 100, 48.06945661827922]
[3209, 100, 48.130927201360464]
[3210, 100, 48.18716675043106]
[3211, 100, 48.2511339969933]
[3212, 100, 48.31212853640318]
[3213, 100, 48.379565984010696]
[3214, 100, 48.43092433363199]
[3215, 100, 48.49444201588631]
[3216, 100, 48.5536735355854]
[3217, 100, 48.61568768322468]
[3218, 100, 48.66572076827288]
[3219, 100, 48.731487549841404]
[322

[3451, 100, 62.75832523405552]
[3452, 100, 62.81552608311176]
[3453, 100, 62.88101939857006]
[3454, 100, 62.933812115341425]
[3455, 100, 63.000504832714796]
[3456, 100, 63.05643733218312]
[3457, 100, 63.12083724886179]
[3458, 100, 63.17584225162864]
[3459, 100, 63.241216130554676]
[3460, 100, 63.30022321641445]
[3461, 100, 63.36304695159197]
[3462, 100, 63.41682346537709]
[3463, 100, 63.47968506813049]
[3464, 100, 63.53733883053064]
[3465, 100, 63.60764825344086]
[3466, 100, 63.66064063459635]
[3467, 100, 63.72373051568866]
[3468, 100, 63.78466038405895]
[3469, 100, 63.84788494929671]
[3470, 100, 63.90143579989672]
[3471, 100, 63.9667399674654]
[3472, 100, 64.02326948195696]
[3473, 100, 64.08714732900262]
[3474, 100, 64.14205431938171]
[3475, 100, 64.20845307037234]
[3476, 100, 64.2672847546637]
[3477, 100, 64.33241415396333]
[3478, 100, 64.38369611650705]
[3479, 100, 64.44716504961252]
[3480, 100, 64.50956058502197]
[3481, 100, 64.57096626609564]
[3482, 100, 64.62258613482118]
[3483, 

In [7]:
data=[]
import time
start=time.time()/60
for m in [3611..4818]:
    poly=H6(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no12.txt','wb') # H6 series (date typo)
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3611, 100, 0.06754789128899574]
[3612, 100, 0.1321321204304695]
[3613, 100, 0.19740647077560425]
[3614, 100, 0.25335483998060226]
[3615, 100, 0.32399775460362434]
[3616, 100, 0.38034043833613396]
[3617, 100, 0.447005070745945]
[3618, 100, 0.5025559067726135]
[3619, 100, 0.5705549530684948]
[3620, 100, 0.6311130747199059]
[3621, 100, 0.6992927379906178]
[3622, 100, 0.7514930367469788]
[3623, 100, 0.8163117244839668]
[3624, 100, 0.8768319375813007]
[3625, 100, 0.9463965408504009]
[3626, 100, 1.0023462250828743]
[3627, 100, 1.0704915076494217]
[3628, 100, 1.1296232715249062]
[3629, 100, 1.194605104625225]
[3630, 100, 1.2530682571232319]
[3631, 100, 1.3183243721723557]
[3632, 100, 1.3763524889945984]
[3633, 100, 1.4470739364624023]
[3634, 100, 1.4990081563591957]
[3635, 100, 1.566419754177332]
[3636, 100, 1.6287408545613289]
[3637, 100, 1.6943323016166687]
[3638, 100, 1.7485819570720196]
[3639, 100, 1.8172882720828056]
[3640, 100, 1.8803583048284054]
[3641, 100, 1.9458383731544018]
[3642,

[3872, 100, 16.281093038618565]
[3873, 100, 16.345741719007492]
[3874, 100, 16.39813617244363]
[3875, 100, 16.463348872959614]
[3876, 100, 16.525404654443264]
[3877, 100, 16.587060820311308]
[3878, 100, 16.640725038945675]
[3879, 100, 16.706202421337366]
[3880, 100, 16.766408056020737]
[3881, 100, 16.829253435134888]
[3882, 100, 16.883745055645704]
[3883, 100, 16.948335137218237]
[3884, 100, 17.005366403609514]
[3885, 100, 17.075760822743177]
[3886, 100, 17.12889313697815]
[3887, 100, 17.1955690048635]
[3888, 100, 17.255898851901293]
[3889, 100, 17.320457853376865]
[3890, 100, 17.373295336961746]
[3891, 100, 17.440203856676817]
[3892, 100, 17.500630851835012]
[3893, 100, 17.56453152373433]
[3894, 100, 17.620195522904396]
[3895, 100, 17.68738852068782]
[3896, 100, 17.745737601071596]
[3897, 100, 17.811708386987448]
[3898, 100, 17.86446723714471]
[3899, 100, 17.929608955979347]
[3900, 100, 17.994555972516537]
[3901, 100, 18.05801047384739]
[3902, 100, 18.110472552478313]
[3903, 100, 18.1

[4132, 100, 32.272746574133635]
[4133, 100, 32.33636713773012]
[4134, 100, 32.39387132227421]
[4135, 100, 32.458861988037825]
[4136, 100, 32.51875740289688]
[4137, 100, 32.58830489218235]
[4138, 100, 32.64084515720606]
[4139, 100, 32.70584397017956]
[4140, 100, 32.77031998708844]
[4141, 100, 32.83585365861654]
[4142, 100, 32.88909438997507]
[4143, 100, 32.95644200593233]
[4144, 100, 33.01448753848672]
[4145, 100, 33.080835085362196]
[4146, 100, 33.13495975360274]
[4147, 100, 33.200726203620434]
[4148, 100, 33.260511353611946]
[4149, 100, 33.32803604006767]
[4150, 100, 33.384107921272516]
[4151, 100, 33.449972320348024]
[4152, 100, 33.51037737354636]
[4153, 100, 33.57359228655696]
[4154, 100, 33.62597718834877]
[4155, 100, 33.695311184972525]
[4156, 100, 33.75318071991205]
[4157, 100, 33.81795918568969]
[4158, 100, 33.87645290419459]
[4159, 100, 33.94102732092142]
[4160, 100, 33.99920551851392]
[4161, 100, 34.066657066345215]
[4162, 100, 34.11902163922787]
[4163, 100, 34.18264363706112]

[4395, 100, 48.5245809070766]
[4396, 100, 48.58401807397604]
[4397, 100, 48.64759771898389]
[4398, 100, 48.702674474567175]
[4399, 100, 48.76775540411472]
[4400, 100, 48.830387741327286]
[4401, 100, 48.897409968078136]
[4402, 100, 48.94997886940837]
[4403, 100, 49.01661993935704]
[4404, 100, 49.07784628495574]
[4405, 100, 49.14460514113307]
[4406, 100, 49.195577435195446]
[4407, 100, 49.263771023601294]
[4408, 100, 49.322586972266436]
[4409, 100, 49.38866985961795]
[4410, 100, 49.44803423807025]
[4411, 100, 49.514924090355635]
[4412, 100, 49.57336000353098]
[4413, 100, 49.640623319894075]
[4414, 100, 49.69311562180519]
[4415, 100, 49.760289669036865]
[4416, 100, 49.81896812468767]
[4417, 100, 49.88526063412428]
[4418, 100, 49.938807275146246]
[4419, 100, 50.00495668873191]
[4420, 100, 50.06832670420408]
[4421, 100, 50.13279240205884]
[4422, 100, 50.18941267207265]
[4423, 100, 50.253072306513786]
[4424, 100, 50.31328925490379]
[4425, 100, 50.38277742266655]
[4426, 100, 50.43549798429012

[4658, 100, 64.88186662271619]
[4659, 100, 64.94938151910901]
[4660, 100, 65.01115255057812]
[4661, 100, 65.07593287155032]
[4662, 100, 65.13408735394478]
[4663, 100, 65.19917447119951]
[4664, 100, 65.25967523828149]
[4665, 100, 65.32987003773451]
[4666, 100, 65.38326785713434]
[4667, 100, 65.44914476945996]
[4668, 100, 65.51208405569196]
[4669, 100, 65.5794347524643]
[4670, 100, 65.63586647436023]
[4671, 100, 65.70279157161713]
[4672, 100, 65.75941377133131]
[4673, 100, 65.82238667085767]
[4674, 100, 65.87929847463965]
[4675, 100, 65.94844010472298]
[4676, 100, 66.00961235538125]
[4677, 100, 66.07682888582349]
[4678, 100, 66.129759054631]
[4679, 100, 66.19492955878377]
[4680, 100, 66.25948974117637]
[4681, 100, 66.32497745379806]
[4682, 100, 66.3777529373765]
[4683, 100, 66.44730936735868]
[4684, 100, 66.50700242444873]
[4685, 100, 66.57341170310974]
[4686, 100, 66.63113105297089]
[4687, 100, 66.69573652371764]
[4688, 100, 66.75457755476236]
[4689, 100, 66.82258060574532]
[4690, 100, 